<a href="https://colab.research.google.com/github/ImVikashKr/Translate/blob/main/EmbeddingUsingHuggingFace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd

In [ ]:
pdf_df = pd.read_csv('/content/drive/MyDrive/pdf_df.csv')

In [ ]:
pdf_df

In [ ]:
pdf_df.dropna(subset=['pdf_text'], inplace=True)

In [ ]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Download necessary NLTK resources
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

# Create stopwords and lemmatizer objects
stopwords = stopwords.words('english')
lemmatizer = WordNetLemmatizer()

# Define function for cleaning and preprocessing text
def clean_text(text):
    # Remove unwanted characters
    text = text.replace('\n', ' ')
    text = text.replace('\r', '')
    text = text.replace('\t', ' ')
    text = re.sub(r'[^\w\s]', '', text)
    
    # Tokenize text into words
    words = nltk.word_tokenize(text)
    
    # Remove stopwords and lemmatize words
    cleaned_words = []
    for word in words:
        if word not in stopwords:
            cleaned_word = lemmatizer.lemmatize(word)
            cleaned_words.append(cleaned_word)
    
    # Join cleaned words into a paragraph
    cleaned_paragraph = ' '.join(cleaned_words)
    
    return cleaned_paragraph

In [ ]:
import numpy as np
import re
import nltk
nltk.download('punkt')

pdf_df['pdf_text'] = pdf_df['pdf_text'].replace(np.nan, '', regex=True)

# Clean and preprocess the 'pdf_text' column
pdf_df['clean_text'] = pdf_df['pdf_text'].apply(lambda x: clean_text(x))

In [ ]:
pdf_df

In [ ]:
!pip install -q transformers

In [ ]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModel

# Load pre-trained tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModel.from_pretrained("bert-base-uncased")

# Define a function to generate embeddings
def generate_embedding(text):
    # Tokenize the text
    tokens = tokenizer(text, padding=True, truncation=True, max_length=512, return_tensors="pt")
    # Generate the embedding
    with torch.no_grad():
        output = model(**tokens)
        embedding = output.last_hidden_state.mean(dim=1).squeeze().tolist()
    return embedding

In [ ]:
# Generate embeddings for each row in the "clean_text" column
pdf_df["embedding"] = pdf_df["clean_text"].apply(generate_embedding)

In [ ]:
pdf_df

,id,pdf,pdf_text,clean_text,embedding
0,014f9e3cd1daf01d8e1868ce4949287a1e86eae4,https://artemis-documents-v1.s3.ap-south-1.ama...,RSLS AJUPITICE L OK AD AL A T PL A TFORM \nApp...,RSLS AJUPITICE L OK AD AL A T PL A TFORM Appli...,"[-0.2871808409690857, 0.07005703449249268, 0.6..."
1,015d5ae762acae92217b4eebf7df1e3d0d3b2c6a,https://artemis-documents-v1.s3.ap-south-1.ama...,S. No. Name Hosting By Performance Department\...,S No Name Hosting By Performance Department 1O...,"[-0.15077708661556244, 0.14911992847919464, 0...."
2,017dbde453ebcc4ac5f8994fef2c1cee65ab848c,https://artemis-documents-v1.s3.ap-south-1.ama...,Test File\n,Test File,"[0.11717420071363449, -0.38490238785743713, -0..."
3,018bca4a36993c10e8e6d9b6f82ee98e59a561c0,https://artemis-documents-v1.s3.ap-south-1.ama...,Online Payment\nAdd Administration Fee\nRemo...,Online Payment Add Administration Fee Remove A...,"[-0.2174089103937149, 0.13876448571681976, 0.4..."
4,01f9c3c9694c9d529cf5a114a02ca5c6c6134fa9,https://artemis-documents-v1.s3.ap-south-1.ama...,MARRIAGE \nCERTIFICATE\nTHIS IS TO CERTIFY THA...,MARRIAGE CERTIFICATE THIS IS TO CERTIFY THAT S...,"[0.04667937010526657, -0.2587149739265442, 0.2..."
...,...,...,...,...,...
922,ff74866fa68286580602b51b100d2f84db3c8fcd,https://artemis-documents-v1.s3.ap-south-1.ama...,A Simple PDF File \n This is a small demonstr...,A Simple PDF File This small demonstration pdf...,"[-0.0366743803024292, 0.06642139703035355, 0.7..."
923,ff792a5cc96b6019590cd19657de41a99009d707,https://artemis-documents-v1.s3.ap-south-1.ama...,From :\nno-r eply@app.jupitice.com\nTo : godaw...,From eplyappjupiticecom To godawarifarmsyahooc...,"[-0.28121381998062134, -0.026187714189291, 0.6..."
924,ff90a3673797177d3e9c79a5a21e299ed773a2bc,https://artemis-documents-v1.s3.ap-south-1.ama...,1 \n COMMONWEALTH OF THE BAHAMAS \nIN THE SUP...,1 COMMONWEALTH OF THE BAHAMAS IN THE SUPREME C...,"[-0.3322094678878784, 0.00733950175344944, 0.4..."
925,ff918298e3bcf1cce14388227ee86442f31a51f4,https://artemis-documents-v1.s3.ap-south-1.ama...,Terms of Reference \nSenior Consultant (Legal...,Terms Reference Senior Consultant Legal Nation...,"[-0.11053542047739029, 0.26765337586402893, 0...."


In [ ]:
for i in pdf_df['embedding']:
  print(i)

[-0.2871808409690857, 0.07005703449249268, 0.6772792339324951, -0.24358169734477997, 0.49863266944885254, 0.06612860411405563, 0.03276955336332321, -0.011349450796842575, 0.1631682813167572, -0.23457741737365723, -0.1696440577507019, -0.12644442915916443, -0.14432646334171295, 0.2662931978702545, -0.020023811608552933, 0.555034875869751, 0.1392834186553955, 0.0122760608792305, -0.3134353756904602, -0.2524639368057251, 1.0306662321090698, 0.02743416279554367, -0.044409189373254776, 0.4086596369743347, 0.6386981010437012, 0.10188358277082443, -0.1269482672214508, -0.4872588515281677, -0.05556212365627289, 0.0875738263130188, 0.11614671349525452, 0.04169183596968651, -0.19569721817970276, -0.3627352714538574, -0.2113499641418457, -0.15609250962734222, -0.3168572187423706, -0.03581999987363815, -0.04500584304332733, 0.44264692068099976, -0.0910118967294693, -0.372808039188385, -0.09756241738796234, 0.050603192299604416, -0.273784875869751, -0.5027412176132202, -0.37262850999832153, 0.08552

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
pdf_df['embedding']

0      [-0.2871808409690857, 0.07005703449249268, 0.6...
1      [-0.15077708661556244, 0.14911992847919464, 0....
2      [0.11717420071363449, -0.38490238785743713, -0...
3      [-0.2174089103937149, 0.13876448571681976, 0.4...
4      [0.04667937010526657, -0.2587149739265442, 0.2...
                             ...                        
922    [-0.0366743803024292, 0.06642139703035355, 0.7...
923    [-0.28121381998062134, -0.026187714189291, 0.6...
924    [-0.3322094678878784, 0.00733950175344944, 0.4...
925    [-0.11053542047739029, 0.26765337586402893, 0....
926    [-0.0366743803024292, 0.06642139703035355, 0.7...
Name: embedding, Length: 862, dtype: object

In [ ]:
pdf_df['embedding'] = pdf_df.embedding.apply(lambda x: np.array(x))

In [ ]:
pdf_df

,id,pdf,pdf_text,clean_text,embedding
0,014f9e3cd1daf01d8e1868ce4949287a1e86eae4,https://artemis-documents-v1.s3.ap-south-1.ama...,RSLS AJUPITICE L OK AD AL A T PL A TFORM \nApp...,RSLS AJUPITICE L OK AD AL A T PL A TFORM Appli...,"[-0.2871808409690857, 0.07005703449249268, 0.6..."
1,015d5ae762acae92217b4eebf7df1e3d0d3b2c6a,https://artemis-documents-v1.s3.ap-south-1.ama...,S. No. Name Hosting By Performance Department\...,S No Name Hosting By Performance Department 1O...,"[-0.15077708661556244, 0.14911992847919464, 0...."
2,017dbde453ebcc4ac5f8994fef2c1cee65ab848c,https://artemis-documents-v1.s3.ap-south-1.ama...,Test File\n,Test File,"[0.11717420071363449, -0.38490238785743713, -0..."
3,018bca4a36993c10e8e6d9b6f82ee98e59a561c0,https://artemis-documents-v1.s3.ap-south-1.ama...,Online Payment\nAdd Administration Fee\nRemo...,Online Payment Add Administration Fee Remove A...,"[-0.2174089103937149, 0.13876448571681976, 0.4..."
4,01f9c3c9694c9d529cf5a114a02ca5c6c6134fa9,https://artemis-documents-v1.s3.ap-south-1.ama...,MARRIAGE \nCERTIFICATE\nTHIS IS TO CERTIFY THA...,MARRIAGE CERTIFICATE THIS IS TO CERTIFY THAT S...,"[0.04667937010526657, -0.2587149739265442, 0.2..."
...,...,...,...,...,...
922,ff74866fa68286580602b51b100d2f84db3c8fcd,https://artemis-documents-v1.s3.ap-south-1.ama...,A Simple PDF File \n This is a small demonstr...,A Simple PDF File This small demonstration pdf...,"[-0.0366743803024292, 0.06642139703035355, 0.7..."
923,ff792a5cc96b6019590cd19657de41a99009d707,https://artemis-documents-v1.s3.ap-south-1.ama...,From :\nno-r eply@app.jupitice.com\nTo : godaw...,From eplyappjupiticecom To godawarifarmsyahooc...,"[-0.28121381998062134, -0.026187714189291, 0.6..."
924,ff90a3673797177d3e9c79a5a21e299ed773a2bc,https://artemis-documents-v1.s3.ap-south-1.ama...,1 \n COMMONWEALTH OF THE BAHAMAS \nIN THE SUP...,1 COMMONWEALTH OF THE BAHAMAS IN THE SUPREME C...,"[-0.3322094678878784, 0.00733950175344944, 0.4..."
925,ff918298e3bcf1cce14388227ee86442f31a51f4,https://artemis-documents-v1.s3.ap-south-1.ama...,Terms of Reference \nSenior Consultant (Legal...,Terms Reference Senior Consultant Legal Nation...,"[-0.11053542047739029, 0.26765337586402893, 0...."


In [ ]:
pdf_df['embedding'] = pdf_df.embedding.apply(lambda x: x.reshape(1, -1))

In [ ]:
# Get user input
query = "I bought land from my father my brother is a witness to the sale deed."

query_embedding = generate_embedding(query)

In [ ]:
query_embedding

[0.10367854684591293,
 -0.0885733962059021,
 0.3621864914894104,
 -0.09081132709980011,
 0.4853134751319885,
 -0.09865765273571014,
 -0.3187771439552307,
 0.47162532806396484,
 0.15767447650432587,
 -0.06948760151863098,
 0.3556663393974304,
 -0.23268301784992218,
 -0.06436166912317276,
 0.30858173966407776,
 -0.3366147577762604,
 -0.0996205285191536,
 0.29291269183158875,
 -0.01950572058558464,
 -0.10186488926410675,
 0.3646341860294342,
 0.08222991973161697,
 -0.023863011971116066,
 -0.17407557368278503,
 0.7492682933807373,
 0.4668482542037964,
 0.42241716384887695,
 -0.14893187582492828,
 0.2627025246620178,
 -0.33231380581855774,
 -0.25303009152412415,
 0.06716925650835037,
 0.09248575568199158,
 -0.05011901631951332,
 -0.08942528069019318,
 -0.1663169115781784,
 -0.4706152379512787,
 -0.24325145781040192,
 -0.03610192984342575,
 -0.6512296795845032,
 0.3432973623275757,
 -0.4532736539840698,
 -0.08530920743942261,
 -0.07835114002227783,
 -0.02885717898607254,
 -0.0873033106327056

In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# convert lists of embeddings to NumPy arrays
query_embedding = np.array(query_embedding)

# reshape embeddings to 2D arrays
pdf = pdf_df.copy()
query_embedding = query_embedding.reshape(1, -1)

# compute cosine similarities
pdf['similarity'] = pdf.embedding.apply(lambda x: cosine_similarity(x, query_embedding)[0][0])
pdf = pdf.sort_values('similarity', ascending=False)

In [ ]:
pdf

,id,pdf,pdf_text,clean_text,embedding,similarity
630,afdc5c19052747217e74c467a43027d08c716ff4,https://artemis-documents-v1.s3.amazonaws.com/...,"CURRICULUM VITAE\nAMANDEEP SINGH\n# 20/2, Old ...",CURRICULUM VITAE AMANDEEP SINGH 202 Old Kasaul...,"[[0.0072785927914083, 0.1527795046567917, 0.32...",0.625480
871,f22ee134b57f673447906c138697fe99c70ba37f,https://artemis-documents-v1.s3.ap-south-1.ama...,"CURRICULUM VITAE\nAMANDEEP SINGH\n# 20/2, Old ...",CURRICULUM VITAE AMANDEEP SINGH 202 Old Kasaul...,"[[0.0072785927914083, 0.1527795046567917, 0.32...",0.625480
899,fb651fac92e80c58bbcde63678cdf9c68488cdae,https://artemis-documents-v1.s3.ap-south-1.ama...,SIGNING OF STATEMENT BY MEDIATOR\n \nDetails ...,SIGNING OF STATEMENT BY MEDIATOR Details party...,"[[-0.04742783308029175, 0.09526291489601135, 0...",0.625338
253,4543c51ed9542bf5468019df75791c8fe8787b50,https://artemis-documents-v1.s3.ap-south-1.ama...,SIGNING OF STATEMENT BY MEDIATOR\n \nDetails ...,SIGNING OF STATEMENT BY MEDIATOR Details party...,"[[-0.04742783308029175, 0.09526291489601135, 0...",0.625338
404,6c123bffec89a76a5d23e793889b3eaf56d42a7e,https://artemis-documents-v1.s3.amazonaws.com/...,Please\tcomplete\tthis\tform\tafter\treviewing...,Please complete form reviewing Rule 31 ADR Cha...,"[[-0.2559760808944702, 0.23890984058380127, 0....",0.617218
...,...,...,...,...,...,...
451,7b3ffbbbf25494877baeeebe4d96562d71207b09,https://artemis-documents-v1.s3.ap-south-1.ama...,授权委托书\n（法人或其他组织使用）\n \n委托单位：\n法定代表人： 职务：...,授权委托书 法人或其他组织使用 委托单位 法定代表人 职务 受委托人 １姓 名 工作单位 名...,"[[-0.17591699957847595, 0.8561296463012695, 0....",0.269478
885,f70af72c55c26d697b30cf50c25c40353b1cfde8,https://artemis-documents-v1.s3.ap-south-1.ama...,授权委托书\n（法人或其他组织使用）\n \n委托单位：\n法定代表人： 职务：...,授权委托书 法人或其他组织使用 委托单位 法定代表人 职务 受委托人 １姓 名 工作单位 名...,"[[-0.17591699957847595, 0.8561296463012695, 0....",0.269478
714,c839bc55179f0da10e781916ef00a6cfef9e5133,https://artemis-documents-v1.s3.ap-south-1.ama...,授权委托书\n（法人或其他组织使用）\n \n委托单位：\n法定代表人： 职务：...,授权委托书 法人或其他组织使用 委托单位 法定代表人 职务 受委托人 １姓 名 工作单位 名...,"[[-0.17591699957847595, 0.8561296463012695, 0....",0.269478
662,ba4568a92b2842827db9aa27c14e649087546610,https://artemis-documents-v1.s3.ap-south-1.ama...,कककक कककक कककक कक कक कककककक कक ककककक कक कककककक...,कककक कककक कककक कक कक कककककक कक ककककक कक कककककक...,"[[-0.1315026432275772, -0.14216525852680206, -...",0.233177


In [ ]:
for i in pdf['pdf'].head(5):
  print(f"Links of recommended pdfs are: {i}")

Links of recommended pdfs are: https://artemis-documents-v1.s3.amazonaws.com/58e30841d5e1e28b224c61b9ec26f7553defd1bcbd481b44c9f0289eaecab475.pdf
Links of recommended pdfs are: https://artemis-documents-v1.s3.ap-south-1.amazonaws.com/58e30841d5e1e28b224c61b9ec26f7553defd1bcbd481b44c9f0289eaecab475.pdf
Links of recommended pdfs are: https://artemis-documents-v1.s3.ap-south-1.amazonaws.com/926ef851607cf793b444d022ecb945b4978505da7bbecc497c105b921621e30d.pdf
Links of recommended pdfs are: https://artemis-documents-v1.s3.ap-south-1.amazonaws.com/6db16402264e62c6591e1473b661012650fda03e8d9fe04f156aad691328695a.pdf
Links of recommended pdfs are: https://artemis-documents-v1.s3.amazonaws.com/d3d9243f660b7698775c1cb2d90c573713326106d43e3acc5fd30456ea3c51a0.pdf


In [ ]:
def recommend_pdfs():

    query = input("Enter your PDF recommendation query: ")

    # generate embedding for query
    query_embedding = generate_embedding(query)
    
    # convert lists of embeddings to NumPy arrays
    query_embedding = np.array(query_embedding)

    # reshape embeddings to 2D arrays
    query_embedding = query_embedding.reshape(1, -1)

    df = pdf_df.copy()

    # compute cosine similarities
    df['similarity'] = df.embedding.apply(lambda x: cosine_similarity(x, query_embedding)[0][0])

    # sort by similarity score
    df = df.sort_values('similarity', ascending=False)

    # return top 5 recommendations
    return df['pdf'].head(5)


In [ ]:
pdfs = recommend_pdfs()

In [ ]:
for i in pdfs:
  print(i)